# Loading packages and settings

In [1]:
#installing wetterdienst API package
!pip install wetterdienst

     |████████████████████████████████| 186 kB 7.5 MB/s 
     |████████████████████████████████| 96 kB 3.7 MB/s 
     |████████████████████████████████| 118 kB 49.2 MB/s 
     |████████████████████████████████| 132 kB 39.9 MB/s 
     |████████████████████████████████| 893 kB 58.5 MB/s 
     |████████████████████████████████| 128 kB 52.7 MB/s 
     |████████████████████████████████| 64 kB 4.0 MB/s 
     |████████████████████████████████| 44 kB 3.5 MB/s 
     |████████████████████████████████| 43.0 MB 1.3 MB/s 
     |████████████████████████████████| 204 kB 62.5 MB/s 
     |████████████████████████████████| 6.4 MB 38.6 MB/s 
     |████████████████████████████████| 1.1 MB 41.6 MB/s 
     |████████████████████████████████| 288 kB 69.6 MB/s 
     |████████████████████████████████| 144 kB 76.1 MB/s 
     |████████████████████████████████| 271 kB 78.9 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 47 kB 5.4 MB/s 
  Created wheel for measu

In [2]:
#installing geopandas
#install geopython libraries
!apt install gdal-bin python-gdal python3-gdal
#install python3-rtree - Geopandas requirement
!apt install python3-rtree 
#install geopandas
!pip install git+git://github.com/geopandas/geopandas.git
#install descartes - Geopandas requirement
!pip install descartes

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (1,590 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

In [3]:
!pip install --upgrade geopandas

!pip install --upgrade pyshp

!pip install --upgrade shapely

!pip install --upgrade descartes

     |████████████████████████████████| 1.0 MB 7.1 MB/s 
     |████████████████████████████████| 6.3 MB 45.1 MB/s 
     |████████████████████████████████| 16.7 MB 145 kB/s 
     |████████████████████████████████| 44 kB 2.3 MB/s 


In [4]:
#importing packages and classes from API package

import pandas as pd
from wetterdienst.provider.dwd.observation import DwdObservationRequest
from wetterdienst import Settings
from google.colab import drive
import geopandas as gpd

In [5]:
#Changing settings of wetterdienst

Settings.tidy = True #default, tidy data
Settings.humanize = True #default, humanized parameters
Settings.si_units = False #DON'T convert values to SI units. For original units, see: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/DESCRIPTION_obsgermany_climate_10min_tu_historical_en.pdf

In [6]:
#packages for the projection
from functools import partial
from pyproj import Proj, transform
from shapely.ops import transform
from shapely.geometry import Point
import numpy as np

# Weather databases download

Downloading all datapoints for 3 weather databases: temperature, precipitation and visibility. All data outputs saved at data/raw.


## Temperature

In [ ]:
#air temperature database, 1 hour granularity, all stations
#for list of stations, see: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/zehn_min_tu_Beschreibung_Stationen.txt 

temperature = DwdObservationRequest(
    parameter=["air_temperature"],
    resolution="hourly",
    start_date="2018-01-01",
    end_date="2021-01-01"
).filter_by_station_id(station_id=["0400", "0410","0420", "0430", "0433"]) #Berlin-Buch, Berlin-Kaniswall, Marzahn, Tegel, Tempelhof (Alexanderplatz doesn't have data past 2015. Link: )

Dircache located at /root/.cache/wetterdienst


In [ ]:
temperature.df.head(10) #list of stations

,station_id,from_date,to_date,height,latitude,longitude,name,state
45,00400,1991-01-01 00:00:00+00:00,2022-04-08 00:00:00+00:00,60.0,52.6310,13.5021,Berlin-Buch,Berlin
47,00410,2004-05-01 00:00:00+00:00,2020-06-15 00:00:00+00:00,33.0,52.4040,13.7309,Berlin-Kaniswall,Berlin
48,00420,2007-08-01 00:00:00+00:00,2022-04-08 00:00:00+00:00,61.0,52.5447,13.5598,Berlin-Marzahn,Berlin
51,00430,1986-01-01 00:00:00+00:00,2021-05-05 00:00:00+00:00,36.0,52.5644,13.3088,Berlin-Tegel,Berlin
52,00433,1951-01-01 00:00:00+00:00,2022-04-08 00:00:00+00:00,48.0,52.4675,13.4021,Berlin-Tempelhof,Berlin


In [ ]:
#getting all values
temperature_df = temperature.values.all().df #all values for all stations

KeyboardInterrupt: ignored

In [ ]:
#saving data - temperatures
#drive.mount('/content/drive')
path = '/content/drive/My Drive/temperature.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  temperature_df.to_csv(f)

In [ ]:
#saving data - station info
path = '/content/drive/My Drive/temperature_stations.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  temperature.df.to_csv(f)

## Precipitation

In [227]:
#precipitation, 10 minutes granularity, all stations
#for list of stations, see: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/zehn_min_tu_Beschreibung_Stationen.txt 

precipitation = DwdObservationRequest(
    parameter=["precipitation"],
    resolution="minute_10",
    start_date="2017-12-31",
    end_date="2021-01-01"
).filter_by_station_id(station_id=["0400", "0410","0420", "0430", "0433"])

Dircache located at /root/.cache/wetterdienst


KeyboardInterrupt: ignored

In [ ]:
precipitation.df.head()

In [ ]:
precipitation_df = precipitation.values.all().df #all values for all stations

In [ ]:
precipitation_df.info()

In [ ]:
#save precipitation data

path = '/content/drive/My Drive/precipitation.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  precipitation_df.to_csv(f)

  #saving data - station info
path = '/content/drive/My Drive/precipitation_stations.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  precipitation.df.to_csv(f)

## Visibility

In [376]:
#visibility, 1 hour granularity, all stations
#for list of stations, see: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/zehn_min_tu_Beschreibung_Stationen.txt 

visibility = DwdObservationRequest(
    parameter=["visibility"],
    resolution="hourly",
    start_date="2018-01-01",
    end_date="2021-01-01"
).filter_by_station_id(station_id=["0400", "0410","0420", "0430", "0433"]) #Berlin-Buch, Berlin-Kaniswall, Marzahn, Tegel, Tempelhof (Alexanderplatz doesn't have data past 2015. Link: )

In [377]:
visibility_df = visibility.values.all().df #all values for all stations

KeyboardInterrupt: ignored

In [ ]:
visibility_df.info()

In [ ]:
visibility_df.groupby(by=["parameter", "station_id"]).count() #only station 430 has data

In [ ]:
#save visibility data
path = '/content/drive/My Drive/data/raw/visibility.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  visibility_df.to_csv(f)

  #saving data - station info
path = '/content/drive/My Drive/data/raw/visibility_stations.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  visibility.df.to_csv(f)

# Data recodes and checks

## Temperature

In [423]:
#loading data
drive.mount("/content/drive")
path = '/content/drive/My Drive/data/raw/temperature.csv'
temperature_df = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [424]:
#Now in the clean_cols function
###temperature_df.rename(columns={ temperature_df.columns[0]: "col_id" }, inplace=True)
###temperature_df.set_index("col_id", inplace=True)
###temperature_df['year'] = pd.DatetimeIndex(temperature_df['date']).year
###temperature_df['month'] = pd.DatetimeIndex(temperature_df['date']).month
###temperature_df['day'] = pd.DatetimeIndex(temperature_df['date']).day
###temperature_df['hour'] = pd.DatetimeIndex(temperature_df['date']).hour
###temperature_df = temperature_df[temperature_df["year"] != 2021]
###
###import datetime
###temperature_df.loc[:, "weekday"] = pd.DatetimeIndex(temperature_df['date']).dayofweek
###temperature_df["weekday"] = temperature_df["weekday"] + 1
###temperature_df.loc[:, "time"] = pd.to_datetime(temperature_df[['year', 'month', 'weekday', "hour"]]) #can't create with weekday
###temperature_df["weekday"] = temperature_df[["date"]].dt.dayofweek

In [425]:
#temperature_df["day"].value_counts()

In [426]:
#temperature_df["weekday"].value_counts()

In [427]:
####getting average per weekday - now in clean_cols function
###temperature_day_average = temperature_df.groupby(by=["year", "month", "weekday", "hour", "station_id", "parameter"]).mean()
###temperature_day_average = temperature_day_average.reset_index()
###temperature_day_average[temperature_day_average["year"]==2018]
###temperature_day_average.head(20)

In [428]:
#temperature_day_average.info()

In [429]:
#temperature_df.info()

In [430]:
#temperature_df.head(10)

In [431]:
#temperature_df.groupby(by=["station_id", "year"]).count() #number of non-missing data per station and year

In [432]:
#temperature_df.groupby(by=["parameter"]).count() #values per parameter, hourly has less parameters? mjust humidity and temperature_air_mean_200

In [433]:
#temperature_df.groupby(by=["month", "parameter"]).mean() #checking that means make sense

## Precipitation

In [458]:
#loading data
#drive.mount("/content/drive")
path = '/content/drive/My Drive/data/raw/precipitation.csv'
precipitation_df = pd.read_csv(path)

In [459]:
#precipitation = clean_cols(precipitation_df)

In [460]:
#precipitation.groupby(by=["year", "month", "parameter"]).mean()

In [461]:
#precipitation.groupby(by=["station_id", "parameter"]).count()

In [462]:
#precipitation.info()

## Visibility

In [473]:
#load data
path = '/content/drive/My Drive/data/raw/visibility.csv'
visibility = pd.read_csv(path)

In [474]:
visibility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131525 entries, 0 to 131524
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  131525 non-null  int64  
 1   station_id  131525 non-null  int64  
 2   dataset     131525 non-null  object 
 3   parameter   131525 non-null  object 
 4   date        131525 non-null  object 
 5   value       26300 non-null   float64
 6   quality     26300 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 7.0+ MB


In [475]:
visibility.head()

,Unnamed: 0,station_id,dataset,parameter,date,value,quality
0,0,400,visibility,visibility,2018-01-01 00:00:00+00:00,NaN,NaN
1,1,400,visibility,visibility,2018-01-01 01:00:00+00:00,NaN,NaN
2,2,400,visibility,visibility,2018-01-01 02:00:00+00:00,NaN,NaN
3,3,400,visibility,visibility,2018-01-01 03:00:00+00:00,NaN,NaN
4,4,400,visibility,visibility,2018-01-01 04:00:00+00:00,NaN,NaN


In [476]:
#checking the number of data points
vis = visibility.copy()
vis["year"] = pd.DatetimeIndex(vis["date"]).year

vis[vis["station_id"] == 430].groupby(by = vis["year"]).count()

,Unnamed: 0,station_id,dataset,parameter,date,value,quality,year
year,,,,,,,,
2018,8760,8760,8760,8760,8760,8760,8760,8760
2019,8760,8760,8760,8760,8760,8760,8760,8760
2020,8784,8784,8784,8784,8784,8779,8779,8784
2021,1,1,1,1,1,1,1,1


## Collision data and weather stations

In [200]:
#collision data
#drive.mount("/content/drive")
#path = '/content/drive/MyDrive/data/output/collisions_shp.dbf'
#collision = gpd.read_file(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Weather stations

In [31]:
#weather stations - temperature dataset
path = '/content/drive/My Drive/data/raw/temperature_stations.csv'
stations = pd.read_csv(path)

In [32]:
#weather stations - precipitation dataset
path = '/content/drive/My Drive/data/raw/precipitation_stations.csv'
stations_precipitation = pd.read_csv(path)

In [33]:
#weather stations - visibility dataset - only one station
#path = "/content/drive/My_Drive/data/raw/visibility_stations.csv"
#stations_visibility = pd.read_csv(path)

FileNotFoundError: ignored

# Data transformations and data extraction

## Functions that are applied to all dataframes

Getting distances from data points to weather stations and preparing weather databases for merge 

In [390]:
def clean_cols(df):
  #cleans data types and creates columns to be used later

  df.rename(columns={df.columns[0]: "col_id" }, inplace=True)
  df.set_index("col_id", inplace=True)
  
  #recoding date to match colision dataframe
  df['year'] = pd.DatetimeIndex(df['date']).year
  df['month'] = pd.DatetimeIndex(df['date']).month
  df['hour'] = pd.DatetimeIndex(df['date']).hour

  #getting day of the week to calculate average later
  df.loc[:, "weekday"] = pd.DatetimeIndex(df['date']).dayofweek #getting day of week from date (Monday = 0, Sunday = 6)
  df.loc[:, "weekday"] = df["weekday"] + 2 #setting day of week to match collision dataframe where Sunday == 1
  df.loc[df["weekday"] == 8, "weekday"] = 1

  #ADDED! Check if OK
  df = df.rename(columns={"weekday": "day"})

  df = df[df["year"] != 2021] #filter 2021 out

  return df

In [327]:
def get_average(df):

  #takes clean columns of weather dataframe and returns the average value of weather per hour of the day of the month of the year

  #calculating the average value of weather per hour of the day of the month
  df = df.groupby(by=["year", "month", "day", "hour", "station_id", "parameter"]).mean() #WEEKDAY CHANGED TO DAY
  df = df.reset_index()

  return df

In [328]:
def get_distances(collision, stations):

  #takes a clean collision geodataframe and stations csv dataframe, returns a dataframe with distances for every collision in collisions dataset

  #turn stations dataframe to GeoDataframe
  stations = gpd.GeoDataFrame(
    stations, geometry=gpd.points_from_xy(stations.longitude, stations.latitude)) #we just need to define where geometry points are saved
  stations["geometry"] = stations["geometry"].set_crs("EPSG:4326") #set the geometry to latitude/longitude
  
  #set projection to calculate distance in meters. points need to already be set to EPSG:4326 #CHECK WITH 3174
  collision = collision.to_crs(epsg=3149)
  stations = stations.to_crs(epsg=3149)

  #dictionary from stations geodataframe
  points = stations.set_index("station_id").to_dict()["geometry"] #weather stations into dictionary

  #calculate distance
  distance = []
  for key, value in points.items(): #iterrating over dictionary pairs
    d = collision.distance(value).rename(key) #calculating distance from collision dataframe and saving in columns named by the key
    distance.append(d)
  
  return pd.concat(distance, axis=1)

In [329]:
def pivot_weather(df, precipitation=False):

  #setting data types
  df = df.astype("object")

  #creating unique indetifier - using weekday instead of day to create a "timestamp"
  #df = df.rename(columns={"weekday": "day"})
  df["datetime"] = pd.to_datetime(df[['year', 'month', 'day', "hour"]])

  if precipitation==True:
    d = df.pivot_table('value', ["datetime", "station_id", "year", "month", "day", "hour"], 'parameter') #pivoting temperature dataframe, to get weather data
    d = d.reset_index()
  else: 
    #pivoting
    d = df.pivot_table('value', ["datetime", "station_id"], 'parameter') #pivoting temperature dataframe, to get weather data
    d = d.reset_index()

  d["station_id"] = d["station_id"].astype("object") #so that data types are the same for later merge


  return d

In [446]:
def get_temperature(collision, distances, temperature):

  #takes collision dataframe, distances dataframe and pivoted temperature dataframe and outputs temperature and humidity with col index from collision

  collision_d = collision
  collision_d["st_closest"] = distances_full.loc[:, "400":"433"].idxmin(axis=1).astype("object") #geting the station_id of the nearest station
  collision_d["st_2closest"] = distances_full.loc[:, "400":"433"].mask(distances_full.loc[:, "400":"433"].eq(distances_full.min(axis=1), axis=0)).idxmin(axis=1).astype("object") #getting second closest by masking the closest

  #setting datetime for unique identification in collision dataframe
  collision_d = collision_d.rename(columns={"weekday": "day"})
  collision_d["datetime"] = pd.to_datetime(collision_d[['year', 'month', 'day', "hour"]])

  #getting data for the closest and second closest station
  collision_d = collision_d.merge(temperature[["temperature_air_mean_200", "humidity", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_closest"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  collision_d = collision_d.merge(temperature[["temperature_air_mean_200", "humidity", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_2closest"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  
  #filling missing data for the closest station with data from the second closest station
  collision_d["temperature_air_mean_200_x"] = collision_d["temperature_air_mean_200_x"].fillna(collision_d["temperature_air_mean_200_y"])
  collision_d["humidity_x"] = collision_d["humidity_x"].fillna(collision_d["humidity_y"])

  #renaming columns
  collision_d = collision_d.rename(columns={"temperature_air_mean_200_x": "temperature", "humidity_x": "humidity"})

  #return collision_d[["col_id", "objectid", "temperature", "humidity"]]
  return collision_d[["year", "month", "day", "hour", "segment_id", "temperature", "humidity"]] #for negative samples




In [457]:
def get_precipitation(collision, distances, precipitation):

  #takes collision dataframe, distances dataframe and pivoted precipitation dataframe with rolling averages and outputs precipitation hight and duration with col index from collision
  
  collision_d = collision

  #gets closest station for precipitation duration (only two stations with data: 430 & 433)
  collision_d["st_closest_duration"] = distances.loc[:, "430":"433"].idxmin(axis=1).astype("object")
  collision_d["st_2closest_duration"] = distances.loc[:, "430":"433"].mask(distances.loc[:, "430":"433"].eq(distances.min(axis=1), axis=0)).idxmin(axis=1).astype("object")
  
  #gets closest station for precipitation hight
  collision_d["st_closest_height"] = distances.loc[:, "400":"433"].idxmin(axis=1).astype("object") #geting the station_id of the nearest station
  collision_d["st_2closest_height"] = distances.loc[:, "400":"433"].mask(distances.loc[:, "400":"433"].eq(distances.min(axis=1), axis=0)).idxmin(axis=1).astype("object") #getting second closest by masking the closest

  #setting datetime for unique identification in collision dataframe
  collision_d = collision_d.rename(columns={"weekday": "day"})
  collision_d["datetime"] = pd.to_datetime(collision_d[['year', 'month', 'day', "hour"]])
  precipitation["datetime"] = pd.to_datetime(precipitation[['year', 'month', 'day', "hour"]])

  #collision_d["hour_before"] = collision_d["datetime"] - pd.Timedelta(1, unit="h")

  #getting precipitation duration
  collision_d = collision_d.merge(precipitation[["prec_duration", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_closest_duration"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  collision_d = collision_d.merge(precipitation[["prec_duration", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_2closest_duration"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])

  #getting precipitation height for the closest and second closest station
  collision_d = collision_d.merge(precipitation[["prec_height", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_closest_height"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  collision_d = collision_d.merge(precipitation[["prec_height", "datetime", "station_id"]], how = "left", left_on=["datetime", "st_2closest_height"], right_on=["datetime", "station_id"]).drop(columns = ['station_id'])
  
  #filling missing data for precipitation duration and height for the closest station with data from the second closest station
  collision_d["prec_duration_x"] = collision_d["prec_duration_x"].fillna(collision_d["prec_duration_y"])
  #collision_d["prec_duration_x"] = collision_d["prec_duration_x"].fillna(collision_d["prec_duration_x"].mean()) #fill the rest of the missing with the mean
  collision_d["prec_height_x"] = collision_d["prec_height_x"].fillna(collision_d["prec_height_y"])
  
  #renaming columns
  collision_d = collision_d.rename(columns={"prec_duration_x":"prec_duration","prec_height_x": "prec_height"})

  #return collision_d[["col_id", "objectid", "prec_duration", "prec_height"]]
  return collision_d[["year", "month", "day", "hour", "segment_id", "prec_duration", "prec_height"]] #for negative samples


In [482]:
def get_visibility(collision, visibility):

  #takes collision dataframe and pivoted visibility dataframe and outputs visibility with col index from collision
  #we don't need distances from weather station as there is only one station 

  collision_d = collision

  #setting datetime for unique identification in collision dataframe
  collision_d = collision_d.rename(columns={"weekday": "day"})
  collision_d["datetime"] = pd.to_datetime(collision_d[['year', 'month', 'day', "hour"]])

  #getting visibility
  collision_d["hour_before"] = collision_d["datetime"] - pd.Timedelta(1, unit="h") #setting a column for an hour before to fill missing data

  collision_d = collision_d.merge(visibility[["visibility", "datetime"]], how = "left", left_on="datetime", right_on="datetime").drop(columns = ["datetime"]) #getting data for the hour of the collision
  collision_d = collision_d.merge(visibility[["visibility", "datetime"]], how = "left", left_on="hour_before", right_on="datetime").drop(columns = ["datetime"]) #getting data for hour before

  collision_d["visibility_x"] = collision_d["visibility_x"].fillna(collision_d["visibility_y"]) #filling missing data

  collision_d = collision_d.rename(columns={"visibility_x":"visibility"})

  #return collision_d[["col_id", "objectid", "visibility"]]
  return collision_d[["year", "month", "day", "hour", "segment_id", "visibility"]]


## Temperature

In [367]:
collision.head()

,col_id,objectid,land,district,LOR,street,LOR_ab_202,year,month,hour,...,LINREFY,XGCSWGS84,YGCSWGS84,geometry,st_closest,st_2closest,st_closest_duration,st_2closest_duration,st_closest_height,st_2closest_height
0,0,112695,11,2,2050602,Samariterviertel,2500729,2018,1,15,...,"5827580,738",13.475018,52.513597,POINT (13.47502 52.51360),420,433,433,433,420,433
1,1,112705,11,12,12304314,Ziekowstraße/Freie Scholle,12500824,2018,1,11,...,"5835009,589",13.291022,52.587259,POINT (13.29102 52.58726),430,400,430,433,430,400
2,2,112726,11,2,2040501,Barnimkiez,2400520,2018,1,9,...,"5828733,523",13.420578,52.526019,POINT (13.42058 52.52602),433,430,433,430,433,430
3,3,112737,11,7,7020202,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,...,"5823525,402",13.348288,52.481844,POINT (13.34829 52.48184),433,430,433,430,433,430
4,4,112747,11,3,3020209,Niederschönhausen,3200206,2018,1,15,...,"5835047,26",13.403228,52.583472,POINT (13.40323 52.58347),430,400,430,433,430,400


In [368]:
temperature_clean = clean_cols(temperature_df)

In [242]:
temperature_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263040 entries, 400 to 433
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   dataset    263040 non-null  object 
 1   parameter  263040 non-null  object 
 2   date       263040 non-null  object 
 3   value      231341 non-null  float64
 4   quality    231341 non-null  float64
 5   year       263040 non-null  int64  
 6   month      263040 non-null  int64  
 7   hour       263040 non-null  int64  
 8   weekday    263040 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 20.1+ MB


In [369]:
temperature_avg = get_average(temperature_clean)

In [371]:
temperature_avg["day"].value_counts()

1    8640
2    8640
3    8640
4    8640
5    8640
6    8640
7    8640
Name: day, dtype: int64

In [372]:
#for temperature extraction
distances_full = get_distances(collision, stations)

In [373]:
#pivoted dataframe for temperature
pivot = pivot_weather(temperature_avg)

In [374]:
collision_temperature = get_temperature(collision, distances_full, pivot)

In [375]:
collision_temperature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38851 entries, 0 to 38850
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   col_id       38851 non-null  int64  
 1   objectid     38851 non-null  object 
 2   temperature  38851 non-null  float64
 3   humidity     38851 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.5+ MB


In [226]:
collision_temperature.head(30)

,col_id,objectid,temperature,humidity
0,0,112695,5.940,83.60
1,1,112705,4.600,65.40
2,2,112726,2.980,79.60
3,3,112737,3.780,69.20
4,4,112747,6.200,84.60
5,5,112770,2.800,81.80
6,6,112792,3.620,70.40
7,7,112805,5.300,87.25
8,8,112820,5.060,60.20
9,9,112837,2.275,84.75


In [50]:
####saving data - temperatures
####drive.mount('/content/drive')
###path = '/content/drive/My Drive/data/output/collision_temperature.csv'
###
###with open(path, 'w', encoding = 'utf-8-sig') as f:
###  collision_temperature.to_csv(f)

## Precipitation

In [352]:
precipitation_clean = clean_cols(precipitation_df)

In [353]:
precipitation_clean.head()

,station_id,dataset,parameter,date,value,quality,year,month,hour,day
col_id,,,,,,,,,,
0,400,precipitation,precipitation_duration,2017-12-31 00:00:00+00:00,NaN,NaN,2017,12,0,1
1,400,precipitation,precipitation_duration,2017-12-31 00:10:00+00:00,NaN,NaN,2017,12,0,1
2,400,precipitation,precipitation_duration,2017-12-31 00:20:00+00:00,NaN,NaN,2017,12,0,1
3,400,precipitation,precipitation_duration,2017-12-31 00:30:00+00:00,NaN,NaN,2017,12,0,1
4,400,precipitation,precipitation_duration,2017-12-31 00:40:00+00:00,NaN,NaN,2017,12,0,1


In [354]:
precipitation_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2369520 entries, 0 to 2369533
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   station_id  int64  
 1   dataset     object 
 2   parameter   object 
 3   date        object 
 4   value       float64
 5   quality     float64
 6   year        int64  
 7   month       int64  
 8   hour        int64  
 9   day         int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 198.9+ MB


In [333]:
#distances for precipitation
distances_prec = get_distances(collision, stations_precipitation)

In [334]:
distances_prec.head()

,400,410,420,430,433
0,16510.840982,26628.036638,8408.130192,15792.831520,8932.539592
1,18891.801271,45314.167261,23566.146806,3521.675757,19169.190398
2,16171.666559,31438.564615,12111.369442,10893.880458,8306.709636
3,24537.139456,34376.585874,20003.483226,11981.401498,4998.623842
4,10675.426083,37464.139664,14343.960655,8438.908809,16158.899622


In [355]:
#pivoted dataframe for precipitation
pivot_prec = pivot_weather(precipitation_clean, precipitation=True)

In [261]:
#saving pivot as it takes long
path = '/content/drive/My Drive/precipitation_pivot.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  pivot_prec.to_csv(f)

In [ ]:
#opening pivot from drive

path = '/content/drive/My Drive/precipitation_pivot.csv'
pivot_prec = pd.read_csv(path)

In [356]:
pivot_prec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27000 entries, 0 to 26999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   datetime                    27000 non-null  datetime64[ns]
 1   station_id                  27000 non-null  object        
 2   year                        27000 non-null  int64         
 3   month                       27000 non-null  int64         
 4   day                         27000 non-null  int64         
 5   hour                        27000 non-null  int64         
 6   precipitation_duration      12144 non-null  float64       
 7   precipitation_height        27000 non-null  float64       
 8   precipitation_indicator_wr  12144 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(1)
memory usage: 1.9+ MB


In [357]:
#rolling average for precipitation
pivot_prec_av = pivot_prec.copy()

pivot_prec_av["prec_duration"] = pivot_prec.groupby('station_id')['precipitation_duration'].rolling(10, min_periods=6).mean().reset_index(0,drop=True)
pivot_prec_av["prec_height"] = pivot_prec.groupby('station_id')['precipitation_height'].rolling(10, min_periods=6).mean().reset_index(0,drop=True)

#filtering out all data that is not full hour
#pivot_prec_av["date"] = pd.to_datetime(pivot_prec_av['date'])
#pivot_prec_av["datetime"] = pd.to_datetime(pivot_prec_av['datetime'])
pivot_prec_av = pivot_prec_av.loc[pivot_prec_av["datetime"].dt.minute == 0]
pivot_prec_av = pivot_prec_av.drop(["precipitation_indicator_wr", "precipitation_duration", "precipitation_height"], axis=1)

In [358]:
pivot_prec_av.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27000 entries, 0 to 26999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime       27000 non-null  datetime64[ns]
 1   station_id     27000 non-null  object        
 2   year           27000 non-null  int64         
 3   month          27000 non-null  int64         
 4   day            27000 non-null  int64         
 5   hour           27000 non-null  int64         
 6   prec_duration  12134 non-null  float64       
 7   prec_height    26975 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 1.9+ MB


In [359]:
####calculating average per day of week

precipitation_avg = pivot_prec_av.copy()

#avg for duration
duration_avg = precipitation_avg.groupby(by=["year", "month", "day", "hour", "station_id"])["prec_duration"].mean()
duration_avg = duration_avg.reset_index()

#avg for height
height_avg = precipitation_avg.groupby(by=["year", "month", "day", "hour", "station_id"])["prec_height"].mean()
height_avg = height_avg.reset_index()

#join
precipitation_avg_full = duration_avg.merge(height_avg, how="inner", on=["year", "month", "day", "hour", "station_id"])
#reset_index(0, drop=True)
#precipitation_avg["prec_height_avg"] = precipitation_avg.groupby(by=["year", "month", "day", "hour", "station_id"])["prec_height"].mean().reset_index(0, drop=True)  

In [360]:
precipitation_avg_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27000 entries, 0 to 26999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           27000 non-null  int64  
 1   month          27000 non-null  int64  
 2   day            27000 non-null  int64  
 3   hour           27000 non-null  int64  
 4   station_id     27000 non-null  int64  
 5   prec_duration  12134 non-null  float64
 6   prec_height    26975 non-null  float64
dtypes: float64(2), int64(5)
memory usage: 1.6 MB


In [301]:
###filtered = precipitation_avg_full["station_id"] >= 430 #boolean
###filtered_df = precipitation_avg_full[filtered]
####filtered_df.info()
###bool_index = filtered_df["prec_duration"].isnull()
###filtered_df[bool_index]

,year,month,day,hour,station_id,prec_duration,prec_height
3,2017,12,1,0,430,NaN,NaN
4,2017,12,1,0,433,NaN,NaN
8,2017,12,1,1,430,NaN,NaN
9,2017,12,1,1,433,NaN,NaN
13,2017,12,1,2,430,NaN,NaN
14,2017,12,1,2,433,NaN,NaN
18,2017,12,1,3,430,NaN,NaN
19,2017,12,1,3,433,NaN,NaN
23,2017,12,1,4,430,NaN,NaN
24,2017,12,1,4,433,NaN,NaN


In [362]:
collision_precipitation = get_precipitation(collision, distances_prec, precipitation_avg_full)

In [ ]:
###filtered = collision_precipitation["station_id"] >= 430
###filtered_df = pivot_prec_av[filtered]
###filtered_df.info()
##bool_index = collision_precipitation["prec_duration"].isnull()
##collision_precipitation[bool_index]

,col_id,objectid,prec_duration,prec_height,datetime
23906,23906,199557,NaN,0.0,2019-09-07 21:00:00
23915,23915,199566,NaN,0.0,2019-09-07 21:00:00
23945,23945,199596,NaN,0.0,2019-09-07 20:00:00


In [ ]:
#pivot_prec_av[pivot_prec_av["datetime"] == "2019-09-07 21:00:00"]

,Unnamed: 0,datetime,date,station_id,prec_duration,prec_height
404050,404050,2019-09-07 21:00:00,2019-09-07 21:00:00+00:00,400,NaN,0.000
404051,404051,2019-09-07 21:00:00,2019-09-07 21:00:00+00:00,420,NaN,0.000
404052,404052,2019-09-07 21:00:00,2019-09-07 21:00:00+00:00,430,9.7,0.005


In [ ]:
#pivot_prec_av[pivot_prec_av["datetime"] == "2019-09-07 20:00:00"]

,Unnamed: 0,datetime,date,station_id,prec_duration,prec_height
404032,404032,2019-09-07 20:00:00,2019-09-07 20:00:00+00:00,400,NaN,0.000
404033,404033,2019-09-07 20:00:00,2019-09-07 20:00:00+00:00,420,NaN,0.000
404034,404034,2019-09-07 20:00:00,2019-09-07 20:00:00+00:00,430,7.2,0.001


In [365]:
collision_precipitation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38851 entries, 0 to 38850
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   col_id         38851 non-null  int64  
 1   objectid       38851 non-null  object 
 2   prec_duration  38851 non-null  float64
 3   prec_height    38851 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.5+ MB


In [364]:
collision_precipitation.head()

,col_id,objectid,prec_duration,prec_height
0,0,112695,3.506667,0.044000
1,1,112705,0.313333,0.000133
2,2,112726,2.823333,0.029300
3,3,112737,0.703333,0.000500
4,4,112747,2.620000,0.049900


In [ ]:
path = '/content/drive/My Drive/data/output/collision_precipitation.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  collision_precipitation.to_csv(f)

## Visibility

In [395]:
visibility_clean = clean_cols(visibility)

In [397]:
visibility_clean = visibility_clean[visibility_clean["station_id"] == 430].copy() #only for station 430 that has data

In [398]:
visibility_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26304 entries, 78915 to 105218
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   station_id  26304 non-null  int64  
 1   dataset     26304 non-null  object 
 2   parameter   26304 non-null  object 
 3   date        26304 non-null  object 
 4   value       26299 non-null  float64
 5   quality     26299 non-null  float64
 6   year        26304 non-null  int64  
 7   month       26304 non-null  int64  
 8   hour        26304 non-null  int64  
 9   day         26304 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 2.2+ MB


In [399]:
visibility_avg = get_average(visibility_clean)

In [401]:
#pivoted visibility data
pivot_visibility = pivot_weather(visibility_avg)

In [402]:
pivot_visibility.head()

parameter,datetime,station_id,visibility
0,2018-01-01 00:00:00,430,26250.0
1,2018-01-01 01:00:00,430,25000.0
2,2018-01-01 02:00:00,430,25000.0
3,2018-01-01 03:00:00,430,26000.0
4,2018-01-01 04:00:00,430,19750.0


In [410]:
collision_visibility = get_visibility(collision, pivot_visibility)

In [411]:
collision_visibility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38851 entries, 0 to 38850
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   col_id      38851 non-null  int64  
 1   objectid    38851 non-null  object 
 2   visibility  38851 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ MB


In [ ]:
#saving data
path = '/content/drive/My Drive/data/output/collision_visibility.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  collision_visibility.to_csv(f)

In [ ]:
#geopy gives correct distance, unilike geopandas
dist = geopy.distance.geodesic((13.50210, 52.63100), (13.42725, 52.63382)) #Buch to accident 0
print(dist)

8.28662021473542 km


# Final generation of data

## Collision negative samples tryout

In [412]:
#full collision dataframe - with negative samples
path = "/content/drive/MyDrive/data/raw/negative_samples.csv"
neg_samples = pd.read_csv(path)

In [413]:
#segments midpoints
path = "/content/drive/MyDrive/data/output/road_shp2.shp"
segments = gpd.read_file(path)

In [414]:
neg_samples.info()
#neg_samples["segment_id"] = neg_samples["segment_id"].astype("int") 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643187 entries, 0 to 643186
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    643187 non-null  int64  
 1   hour          643187 non-null  int64  
 2   year          643187 non-null  int64  
 3   month         643187 non-null  int64  
 4   weekday_name  643187 non-null  object 
 5   weekday       643187 non-null  int64  
 6   segment_id    643152 non-null  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 34.4+ MB


In [415]:
segments.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 43110 entries, 0 to 43109
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   segment_id  43110 non-null  int64   
 1   element_nr  43110 non-null  object  
 2   strassensc  43110 non-null  object  
 3   strassenna  43110 non-null  object  
 4   str_bez     1295 non-null   object  
 5   strassenkl  43110 non-null  object  
 6   strassen_1  43110 non-null  object  
 7   strassen_2  43110 non-null  object  
 8   verkehrsri  43110 non-null  object  
 9   bezirk      43001 non-null  object  
 10  stadtteil   43001 non-null  object  
 11  verkehrseb  43110 non-null  int64   
 12  beginnt_be  43110 non-null  object  
 13  endet_bei_  43110 non-null  object  
 14  laenge      43110 non-null  float64 
 15  gueltig_vo  43110 non-null  object  
 16  okstra_id   43110 non-null  object  
 17  length_m    43110 non-null  float64 
 18  geometry    43110 non-null  geometry
d

In [416]:
segments["segment_id"] = segments["segment_id"].astype("float") #for join

In [418]:
neg_samples_geo = neg_samples.merge(segments[["segment_id", "geometry"]], how = "left", on="segment_id")

In [420]:
neg_samples_geo = gpd.GeoDataFrame(neg_samples_geo, 
                                   geometry=neg_samples_geo["geometry"], 
                                   crs = {"init" : "EPSG:4326"})  #we just need to define where geometry points are saved

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [421]:
neg_samples_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 643187 entries, 0 to 643186
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   Unnamed: 0    643187 non-null  int64   
 1   hour          643187 non-null  int64   
 2   year          643187 non-null  int64   
 3   month         643187 non-null  int64   
 4   weekday_name  643187 non-null  object  
 5   weekday       643187 non-null  int64   
 6   segment_id    643152 non-null  float64 
 7   geometry      643152 non-null  geometry
dtypes: float64(1), geometry(1), int64(5), object(1)
memory usage: 44.2+ MB


## Temeperature

In [434]:
temperature_clean = clean_cols(temperature_df)

In [435]:
temperature_avg = get_average(temperature_clean)

In [436]:
distances_full = get_distances(neg_samples_geo, stations)

In [437]:
pivot = pivot_weather(temperature_avg)

In [439]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26880 entries, 0 to 26879
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetime                  26880 non-null  datetime64[ns]
 1   station_id                26880 non-null  object        
 2   humidity                  26880 non-null  float64       
 3   temperature_air_mean_200  26880 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 840.1+ KB


In [440]:
neg_samples_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 643187 entries, 0 to 643186
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   Unnamed: 0    643187 non-null  int64   
 1   hour          643187 non-null  int64   
 2   year          643187 non-null  int64   
 3   month         643187 non-null  int64   
 4   weekday_name  643187 non-null  object  
 5   weekday       643187 non-null  int64   
 6   segment_id    643152 non-null  float64 
 7   geometry      643152 non-null  geometry
 8   st_closest    643152 non-null  object  
 9   st_2closest   643152 non-null  object  
dtypes: float64(1), geometry(1), int64(5), object(3)
memory usage: 54.0+ MB


In [447]:
collision_temperature_neg_samples = get_temperature(neg_samples_geo, distances_full, pivot)

In [448]:
collision_temperature_neg_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643187 entries, 0 to 643186
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   year         643187 non-null  int64  
 1   month        643187 non-null  int64  
 2   day          643187 non-null  int64  
 3   hour         643187 non-null  int64  
 4   segment_id   643152 non-null  float64
 5   temperature  643152 non-null  float64
 6   humidity     643152 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 39.3 MB


In [449]:
collision_temperature_neg_samples.head()

,year,month,day,hour,segment_id,temperature,humidity
0,2018,1,2,0,37063.0,3.28,83.8
1,2018,1,2,2,7097.0,3.16,84.0
2,2018,1,2,2,30880.0,2.96,82.8
3,2018,1,2,6,3942.0,2.86,84.0
4,2018,1,2,9,29466.0,3.06,76.0


In [452]:
collision_temperature_neg_samples["day"].value_counts()

5    92367
3    92063
1    91962
4    91839
6    91681
2    91640
7    91635
Name: day, dtype: int64

In [455]:
collision_temperature_neg_samples.groupby(["month"])["temperature"].describe()

,count,mean,std,min,25%,50%,75%,max
month,,,,,,,,
1,53526.0,3.247722,1.900333,-1.550000,1.885,3.175,4.650,8.420
2,53523.0,3.417937,3.807010,-7.625000,0.625,3.725,6.400,10.880
3,53334.0,5.087114,3.293644,-5.225000,2.540,5.275,7.725,12.125
4,53807.0,12.156284,3.852366,0.550000,9.150,12.250,15.350,20.475
5,53372.0,14.589976,4.257777,3.300000,11.350,14.350,17.200,26.225
6,53798.0,20.473642,3.922660,8.333333,17.375,20.400,22.875,33.525
7,53342.0,20.289146,3.555809,11.500000,17.425,20.100,23.175,28.400
8,53559.0,21.705361,3.603063,9.875000,18.625,21.580,24.760,29.780
9,53609.0,16.181086,3.500313,6.740000,13.375,15.900,18.740,24.475


In [ ]:
#SAVE THE DATA
path = '/content/drive/My Drive/data/output/collision_temperature.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  collision_temperature_neg_samples.to_csv(f)

## Precipitation

In [463]:
precipitation_clean = clean_cols(precipitation_df)

In [464]:
distances_prec = get_distances(neg_samples_geo, stations_precipitation)

In [465]:
pivot_prec = pivot_weather(precipitation_clean, precipitation=True)

In [466]:
#rolling average for precipitation
pivot_prec_av = pivot_prec.copy()

pivot_prec_av["prec_duration"] = pivot_prec.groupby('station_id')['precipitation_duration'].rolling(10, min_periods=6).mean().reset_index(0,drop=True)
pivot_prec_av["prec_height"] = pivot_prec.groupby('station_id')['precipitation_height'].rolling(10, min_periods=6).mean().reset_index(0,drop=True)

pivot_prec_av = pivot_prec_av.loc[pivot_prec_av["datetime"].dt.minute == 0]
pivot_prec_av = pivot_prec_av.drop(["precipitation_indicator_wr", "precipitation_duration", "precipitation_height"], axis=1)

In [467]:
####calculating average per day of week

precipitation_avg = pivot_prec_av.copy()

#avg for duration
duration_avg = precipitation_avg.groupby(by=["year", "month", "day", "hour", "station_id"])["prec_duration"].mean()
duration_avg = duration_avg.reset_index()

#avg for height
height_avg = precipitation_avg.groupby(by=["year", "month", "day", "hour", "station_id"])["prec_height"].mean()
height_avg = height_avg.reset_index()

#join
precipitation_avg_full = duration_avg.merge(height_avg, how="inner", on=["year", "month", "day", "hour", "station_id"])

In [468]:
collision_precipitation = get_precipitation(neg_samples_geo, distances_prec, precipitation_avg_full)

In [469]:
collision_precipitation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643187 entries, 0 to 643186
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   year           643187 non-null  int64  
 1   month          643187 non-null  int64  
 2   day            643187 non-null  int64  
 3   hour           643187 non-null  int64  
 4   segment_id     643152 non-null  float64
 5   prec_duration  643152 non-null  float64
 6   prec_height    643152 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 39.3 MB


In [ ]:
#SAVE THE DATA
path = '/content/drive/My Drive/data/output/collision_precipitation.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  collision_precipitation.to_csv(f)

## Visibility

In [477]:
visibility_clean = clean_cols(visibility)

In [478]:
visibility_clean = visibility_clean[visibility_clean["station_id"] == 430].copy() #only for station 430 that has data

In [479]:
visibility_avg = get_average(visibility_clean)

In [480]:
pivot_visibility = pivot_weather(visibility_avg)

In [483]:
collision_visibility = get_visibility(neg_samples_geo, pivot_visibility)

In [484]:
collision_visibility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643187 entries, 0 to 643186
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   year        643187 non-null  int64  
 1   month       643187 non-null  int64  
 2   day         643187 non-null  int64  
 3   hour        643187 non-null  int64  
 4   segment_id  643152 non-null  float64
 5   visibility  643187 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 34.3 MB


In [ ]:
#SAVE THE DATA
path = '/content/drive/My Drive/data/output/collision_visibility.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  collision_visibility.to_csv(f)